In [1]:
import os
for dirname, _, filenames in os.walk('./data_input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


./data_input\combined_data_1.txt
./data_input\combined_data_2.txt
./data_input\combined_data_3.txt
./data_input\combined_data_4.txt
./data_input\movie_titles.csv
./data_input\probe.txt
./data_input\qualifying.txt
./data_input\README


In [22]:
from datetime import datetime
import os
import random
import matplotlib
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix

from sklearn.decomposition import TruncatedSVD

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.layers import Concatenate, Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2

In [3]:
start = datetime.now()
if not os.path.isfile('data.csv'):
    #read all txt file and store them in one big file
    data = open('data.csv', mode='w')
    
    row = list()
    files = ['./data_input/combined_data_1.txt', './data_input/combined_data_2.txt',
            './data_input/combined_data_3.txt', './data_input/combined_data_4.txt']
    for file in files:
        print('reading ratings from {}...'.format(file))
        with open(file) as f:
            for line in f:
                del row[:]
                line = line.strip()
                if line.endswith(':'):
                    #all are rating
                    movid_id = line.replace(':', '')
                else:
                    row = [x for x in line.split(',')]
                    row.insert(0, movid_id)
                    data.write(','.join(row))
                    data.write('\n')
        print('Done.\n')
    data.close()
print('time taken:', datetime.now() - start)

reading ratings from ./data_input\combined_data_1.txt...
Done.

reading ratings from ./data_input\combined_data_2.txt...
Done.

reading ratings from ./data_input\combined_data_3.txt...
Done.

reading ratings from ./data_input\combined_data_4.txt...
Done.

time taken: 0:03:56.678430


In [15]:
if not os.path.isfile('./data_input/data2.csv'):
    data = open('./data_input/data2.csv', mode='w')

files = ['./data_input/combined_data_1.txt', './data_input/combined_data_2.txt',
            './data_input/combined_data_3.txt', './data_input/combined_data_4.txt']

# Remove the line with movie_id: and add a new column of movie_id
# Combine all data files into a csv file
for file in files:
  print("Opening file: {}".format(file))
  with open(file) as f:
    for line in f:
        line = line.strip()
        if line.endswith(':'):
            movie_id = line.replace(':', '')
        else:
            data.write(movie_id + ',' + line)
            data.write('\n')
data.close()

# Read all data into a pd dataframe
df = pd.read_csv('./data_input/data2.csv', names=['movie_id', 'user_id','rating','date'])

df

Opening file: ./data_input/combined_data_1.txt
Opening file: ./data_input/combined_data_2.txt
Opening file: ./data_input/combined_data_3.txt
Opening file: ./data_input/combined_data_4.txt


,movie_id,user_id,rating,date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03
...,...,...,...,...
100480502,17770,1790158,4,2005-11-01
100480503,17770,1608708,3,2005-07-19
100480504,17770,234275,1,2004-08-07
100480505,17770,255278,4,2004-05-28


In [16]:
print('creating the dataframe from data.csv file..')
df = pd.read_csv('./data_input/data.csv', sep=',', names=['movie_id','user_id','rating','date'])

df.date = pd.to_datetime(df.date)
print('Done.\n')

df

#arranging the rating according to time
# print('sorting the dataframe by date..')
# df.sort_values(by='date', inplace=True)
# print('sorting done.')

creating the dataframe from data.csv file..
Done.



,movie_id,user_id,rating,date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03
...,...,...,...,...
100480502,17770,1790158,4,2005-11-01
100480503,17770,1608708,3,2005-07-19
100480504,17770,234275,1,2004-08-07
100480505,17770,255278,4,2004-05-28


In [6]:
df.head()

,movie,user,rating,date
56431994,10341,510180,4,1999-11-11
9056171,1798,510180,5,1999-11-11
58698779,10774,510180,3,1999-11-11
48101611,8651,510180,2,1999-11-11
81893208,14660,510180,2,1999-11-11


In [7]:
df.describe()

,movie,user,rating
count,1.004805e+08,1.004805e+08,1.004805e+08
mean,9.070915e+03,1.322489e+06,3.604290e+00
std,5.131891e+03,7.645368e+05,1.085219e+00
min,1.000000e+00,6.000000e+00,1.000000e+00
25%,4.677000e+03,6.611980e+05,3.000000e+00
50%,9.051000e+03,1.319012e+06,4.000000e+00
75%,1.363500e+04,1.984455e+06,4.000000e+00
max,1.777000e+04,2.649429e+06,5.000000e+00


In [23]:
lite_rating_df = pd.DataFrame()

group = df.groupby('user_id')['rating'].count()
top_users = group.sort_values(ascending=False)[:10000]

group = df.groupby('movie_id')['rating'].count()
top_movies = group.sort_values(ascending=False)[:2000]

lite_rating_df = df.join(top_users, rsuffix='_r', how='inner', on='user_id')
lite_rating_df = lite_rating_df.join(top_movies, rsuffix='_r', how='inner', on='movie_id')

# Re-name the users and movies for uniform name from 0..2000 and 10000
user_enc = LabelEncoder()
lite_rating_df['user'] = user_enc.fit_transform(lite_rating_df['user_id'].values)
movie_enc = LabelEncoder()
lite_rating_df['movie'] = movie_enc.fit_transform(lite_rating_df['movie_id'].values)

n_movies = lite_rating_df['movie'].nunique()
n_users = lite_rating_df['user'].nunique()

# print(n_movies, n_users)
lite_rating_df

,movie_id,user_id,rating,date,rating_r,rating_r,user,movie
5149,8,1488844,4,2005-05-12,2206,14910,5591,0
5883,8,1227322,5,2005-05-31,4752,14910,4587,0
6183,8,525356,4,2005-08-26,4913,14910,1944,0
6310,8,401047,2,2005-06-15,2119,14910,1465,0
7140,8,883478,5,2005-10-10,3222,14910,3259,0
...,...,...,...,...,...,...,...,...
45663998,8163,778937,3,2005-06-10,1284,12553,2874,914
45668920,8163,1230845,3,2005-02-18,1277,12553,4603,914
45670283,8163,733887,4,2005-01-04,1779,12553,2701,914
45666732,8163,1645937,3,2005-02-01,1106,12553,6176,914


In [26]:
#group = df.groupby('user_id')['rating'].count()
# group
top_users

user_id
305344     17653
387418     17436
2439493    16565
1664010    15813
2118461    14831
           ...  
2222535     1102
1617710     1102
428168      1102
1894721     1102
2119135     1102
Name: rating, Length: 10000, dtype: int64

In [27]:
top_movies

movie_id
5317     232944
15124    216596
14313    200832
15205    196397
1905     193941
          ...  
13941     10385
14283     10379
5957      10377
8545      10371
14738     10368
Name: rating, Length: 2000, dtype: int64

In [10]:
lite_df.to_csv(r'lite_data.csv', index = False)